# Training, Validation, and Testing Data Set Exploration

## Date: 12/29/2017
## Author: Hiro Miyake

1. Introduction
2. Module importation and data generation
3. Splitting the data using the `train_test_split` function
4. Model performance evaluation  
    4.1. Train-test split and model performance evaluation  
    4.2. K-fold and shuffle/split cross validation for model performance evaluation
5. Hyperparameter optimization with cross validation  
    5.1. Separate the hyperparameter optimization and prediction processes  
    5.2. Using the `LogisticRegressionCV` function
6. References

# 1. Introduction

In this notebook, I explore some methods to deal with training, validation, and testing data sets. The most important reason to properly handle these three types of data sets is to prevent overfitting. Overfitting is the situation where your model follows the pecularities of your particular data set too closely and does not generalize to unseen data.

For example, if you have some univariate data set with dependent variable $y$ and independent variable $x$, fitting $y$ to a quadratic function of $x$ would be 'better' than a linear function of $x$, and a cubic function of $x$ would be even 'better' than a quadratic fit. If you think about it, you'll realize that, in the univariate case, if you have $n$ data points, a polynomial function of order $n-1$ would fit the data perfectly. But then it's clear that if another data point comes in from the real world, it is highly unlikely that that point will land exactly on the best fit curve. Therefore, you need to have a way to determine the optimal number of degrees of freedom you need to best describe your data. This optimization is often called the bias-variance tradeoff; too many degrees of freedom and you will have low bias and high variance. Too few degrees of freedom and you will have high bias and low variance. Splitting your data into a training, validation, and testing data sets is one way to systematically find the optimum degrees of freedom. Another approach is a method called cross validation. We will explore both approaches in this notebook.

Before we move on, note that in the above example, the number of degrees of freedom corresponds to a hyperparameter of the model. But once you find an optimum hyperparameter with cross validation, you would then like to use that hyperparameter and estimate the test set performance. This again seems like a good place to use a method such as cross validation. Thus there are essentially two things you could do with cross validation,
1. Hyperparameter optimization,
2. Test set prediction quality.

Ideally, these two things should be done on independent data sets. This is most easily done with the train/validation/test split approach, where you can use the training and validation data sets to find the optimum hyperparameter and model, and then use the test data set to determine the test set performance. As long as there is enough data in the test set, this should be a fairly robust method.

However, if data is abundant and not too "weird", the hyperparameter/model selection and test set performance evaluation could probably be done on the same data set by running two cross validation procedures.

One could imagine other approaches, such as splitting the data first in half, doing cross-validation on the first half to find the optimum model and hyperparameters, and doing cross-validation on the second half to determine test set performance. But this may be being too careful at not overfitting.

This issue between hyperparameter optimization and test set prediction is discussed [here](http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html). They point that using the same data for both purposes will give an overly optimistic test set performance estimate. To avoid this, they suggest using a nested CV.

# 2. Module importation and data generation

First I load the necessary modules.

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import log_loss
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

Then I generate some artificial data. The dependent variable has three values: 0, 1, and 2. There are 10 of 0, 20 of 1, and 3 or 2. I generate both a $33 \times 2$ dataframe and two $33 \times 1$ dataframes. This at least illustrates how we can use the `train_test_split` function with the `stratify` option to ensure the fraction of categorical outcomes is preserved in the different splits.

In [2]:
## Make individual vectors first approach
y = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2]
x = [1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3]
y = pd.DataFrame(y, columns = ['y'])
x = pd.DataFrame(x, columns = ['x'])

## Make dataframe first approach
## data is 33 x 2
## 0 is 10, 1 is 20, 3 is 3
data = {'y': [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2],
        'x': [1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3]}
df = pd.DataFrame(data, columns = ['y', 'x'])
df['y'].value_counts()

1    20
0    10
2     3
Name: y, dtype: int64

For a more realistic classification data set, we can use the [iris data set](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). We can import this from `sklearn` directly. Other data set information for Scikit-learn can be found [here](http://scikit-learn.org/stable/datasets/index.html).

In [3]:
iris = datasets.load_iris() ## this is a dictionary with data and associated info
print iris.keys() ## what are the keys of this dictionary?
print iris['target_names'] ## what are the dependent variable labels?
print iris['target'] ## what are the values of the dependent variable in this data set?
print iris['feature_names'] ## what are the names of the independent variables in this data set?
feature_names = ['sep_l', 'sep_w', 'pet_l', 'pet_w'] ## make column name list to make a dataframe
x = iris.data ## extract the independent variables
y = iris.target ## extract the dependent variables

x = pd.DataFrame(x, columns = feature_names)
y = pd.DataFrame(y, columns = ['y'])
print y['y'].value_counts()

df = pd.concat([y, x], axis = 1)
df.head()

['target_names', 'data', 'target', 'DESCR', 'feature_names']
['setosa' 'versicolor' 'virginica']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
2    50
1    50
0    50
Name: y, dtype: int64


,y,sep_l,sep_w,pet_l,pet_w
0,0,5.1,3.5,1.4,0.2
1,0,4.9,3.0,1.4,0.2
2,0,4.7,3.2,1.3,0.2
3,0,4.6,3.1,1.5,0.2
4,0,5.0,3.6,1.4,0.2


# 3. Splitting the data using the `train_test_split` function

The simplest method to split the data into a training and test data set is to use the `train_test_split` function in Scikit-learn. In the example below, we split the data into 67% for the training data and 33% for the test data. We also use the `stratify` option to make sure the split data sets have the same fraction of target value categories as the original data set.

In [4]:
## Info on train_test_split
## http://scikit-learn.org/stable/modules/cross_validation.html#
## http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
## https://chrisalbon.com/machine-learning/split_data_into_training_and_test_sets.html

## not stratified, i.e., random
#df_train, df_test = train_test_split(df, test_size=0.33, random_state=100)

## stratified
df_train, df_test = train_test_split(df, test_size=0.33, random_state=1000, stratify = df['y'])

## Alternative way to stratify split based on separate independent and dependent variable dataframes
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=100, stratify = y)

Let's confirm that the fraction of categorical outcomes is the same for the original data set and the stratified data set.

In [5]:
print df.shape
print df['y'].value_counts()
print df_train.shape
print df_train['y'].value_counts()
print df_test.shape
print df_test['y'].value_counts()

(150, 5)
2    50
1    50
0    50
Name: y, dtype: int64
(100, 5)
2    34
1    33
0    33
Name: y, dtype: int64
(50, 5)
1    17
0    17
2    16
Name: y, dtype: int64


# 4. Model performance evaluation

We go over two of the most popular, model-independent methods to evaluate model performance on unseen data. These are the train-test split method and the cross validation method. We go over each in turn.

## 4.1. Train-test split and model performance evaluation

Here we do the simplest way to evaluate model performance to avoid overfitting. This is done by splitting the data into a training and testing data set. We fit the model with the training set, and then evaluate its performance on the test set. Note that we are not performing hyperparameter tuning here.

First, we specify a logistic regression model and fit it to the training data.

In [6]:
#model = linear_model.LogisticRegression() ## vanilla logistic regression with liblinear solver
model = linear_model.LogisticRegression(solver = 'sag', multi_class = 'multinomial')
#model = linear_model.LogisticRegression(solver = 'sag', max_iter = 100, multi_class = 'multinomial')
model.fit(df_train.iloc[:, 1:], df_train['y'])

/Users/hiro/anaconda/lib/python2.7/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

With the trained model, we can determine its performance on the training set.

In [7]:
## http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html
## http://www.markhneedham.com/blog/2016/09/14/scikit-learn-first-steps-with-log_loss/
pred_train = model.predict_proba(df_train.iloc[:, 1:])
score = log_loss(df_train['y'], pred_train)
print 'Logloss for the training set: ' + str(score)

Logloss for the training set: 0.163458124386


Now we can apply the model to the test set.

In [8]:
pred_train = model.predict_proba(df_test.iloc[:, 1:])
score = log_loss(df_test['y'], pred_train)
print 'Logloss for the validation set: ' + str(score)

Logloss for the validation set: 0.197083478343


We see that the logloss for the test set is larger than the training set, which is not surprising because the modeling process was specifically designed to minimize the training logloss function.

What we have done with train/test splitting would be generally sufficient for most cases when there are no hyperparameters to tune. Next, we see how we can use cross validation.

## 4.2. K-fold and shuffle/split cross validation for model performance evaluation

K-fold cross validation takes the train-test split further. Rather than splitting the data into 2 sets, a training and test set, K-fold cross validation splits the data set into K equal sized subsets. Then the model is training on K-1 of those subsets and tested on the remaining subset. This is done for each of the K folds to get K cross validation performance values. Because the data is split K times and you can look at the average evaluation metric, your evaluation is not so dependent on the particular split of the data.

The simplest way to do K-fold cross validation in Scikit-learn is to specify the model you want to use (logistic regression in the case below) and give that as an argument to the `cross_val_score` function.

In [9]:
## Follow the example in this closely
## http://scikit-learn.org/stable/modules/cross_validation.html
## http://scikit-learn.org/stable/modules/model_evaluation.html

#model = linear_model.LogisticRegression() ## vanilla logistic regression with liblinear solver
model = linear_model.LogisticRegression(solver = 'sag', multi_class = 'multinomial')
## 5-fold CV, automatic detection of stratification, random seed unset, use logloss, use all cores available
scores = cross_val_score(model, df.iloc[:, 1:], df['y'], scoring = 'neg_log_loss', cv=5, n_jobs = -1)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[-0.1209738  -0.13054553 -0.20925457 -0.19706026 -0.13671033]
Accuracy: -0.16 (+/- 0.07)


If you want more control over your cross validation procedure, e.g., be able to set the random seed for splitting, or specify a shuffle split cross validation (which is supposed to be faster than K-fold CV), you can define a cv function specifically and feed it into the `cross_val_score` function.

In [10]:
## Shuffle split versus K-fold CV
## https://stackoverflow.com/questions/34731421/whats-the-difference-between-kfold-and-shufflesplit-cv
## Apparently shuffle split is faster than K-fold CV for large data sets. This might be because generating a
## K-fold split is more computationally intensive than just splitting the entire data set into one training and
## test data set.

#model = linear_model.LogisticRegression() ## vanilla logistic regression with liblinear solver
model = linear_model.LogisticRegression(solver = 'sag', max_iter = 100, multi_class = 'multinomial')

## CHOOSE ONE BELOW (EITHER K-FOLD OR SHUFFLESPLIT)
## http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
#cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
## http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)

scores = cross_val_score(model, df.iloc[:, 1:], df['y'], scoring = 'neg_log_loss', cv = cv, n_jobs = -1) ## 5-fold CV
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[-0.12968749 -0.13072888 -0.19751855 -0.14649421 -0.16136353]
Accuracy: -0.15 (+/- 0.05)


Now, you can determine a reasonably robust logloss value which should transfer over to unseen test data sets.

# 5. Hyperparameter optimization with cross validation

So far, we have not considered any hyperparameter optimization. It turns out that the `LogisticRegression` function takes a hyperparameter `C` which is related to regularization (note the default regularization method is L2). We can use cross validation to find the optimum value for `C`.

In [11]:
## http://scikit-learn.org/stable/modules/grid_search.html
## It is good practice to "train" the hyperparameters on one part of the data and train the model 
## on the remaining part of the data. See the link below.
## http://scikit-learn.org/stable/modules/grid_search.html#model-selection-development-and-evaluation

## 5.1. Separate the hyperparameter optimization and prediction processes

In the simplest case, we can find the optimum hyperparameter by directly running `GridSearchCV` once. Then we manually input the optimum hyperparameter into the model and do a cross validation to determine the prediction logloss.

Note that `GridSearchCV` behaves just like the estimator (e.g., `LogisticRegression`) it was given. So there is a `fit` function, as well as `predict`, etc.

In [12]:
## Find the best hyperparameter
## http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
## http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html

model = linear_model.LogisticRegression(solver = 'sag', max_iter = 100, multi_class = 'multinomial')
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
tuned_parameters = [{'C': [1, 10, 100, 1000, 10000, 100000]}]
clf = GridSearchCV(model, tuned_parameters, cv=cv, scoring='neg_log_loss')
clf.fit(df.iloc[:, 1:], df['y'])
print clf.best_params_
#print clf.cv_results_
print clf.best_score_

{'C': 100000}
-0.0765629647844


In [13]:
## Find the cross validated logloss using the best hyperparameter

model = linear_model.LogisticRegression(C = 1000, solver = 'sag', max_iter = 100, multi_class = 'multinomial')
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
scores = cross_val_score(model, df.iloc[:, 1:], df['y'], scoring = 'neg_log_loss', cv = cv, n_jobs = -1)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[-0.04654217 -0.03652902 -0.1693998  -0.06420904 -0.06976105]
Accuracy: -0.08 (+/- 0.10)


In [14]:
model.get_params()

{'C': 1000,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'multinomial',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'sag',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In the example [here](http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html), the data is split 50% into a train/validation set for `GridSearchCV` to both optimize the hyperparameters and build the best model, and the other 50% is used as a test set, using the model built with `GridSearchCV`.

## 5.2. Using the `LogisticRegressionCV` function

You can also use the `LogisticRegressionCV` function, which combines logistic regression and cross validation into a single function and [takes advantage](http://scikit-learn.org/stable/modules/grid_search.html#model-specific-cross-validation) of the specific features of logistic regression. However, as mentioned in the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html), "[f]or a multiclass problem, the hyperparameters for each class are computed using the best scores got by doing a one-vs-rest in parallel across all folds and classes. Hence this is not the true multinomial loss." So this may not be the best function if you want full control of the cross validation process.

In [15]:
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
model = linear_model.LogisticRegressionCV(Cs = 10, cv = cv, scoring = 'neg_log_loss',
                                          solver = 'sag', max_iter = 100, multi_class = 'multinomial',
                                          random_state = 0, n_jobs = -1)
model.fit(df_train.iloc[:, 1:], df_train['y'])
print model.Cs_
print model.C_
#print model.scores_
pred_train = model.predict_proba(df_train.iloc[:, 1:])
score = log_loss(df_train['y'], pred_train)
print 'Logloss for the training set: ' + str(score)
pred_train = model.predict_proba(df_test.iloc[:, 1:])
score = log_loss(df_test['y'], pred_train)
print 'Logloss for the validation set: ' + str(score)

[  1.00000000e-04   7.74263683e-04   5.99484250e-03   4.64158883e-02
   3.59381366e-01   2.78255940e+00   2.15443469e+01   1.66810054e+02
   1.29154967e+03   1.00000000e+04]
[ 1291.54966501  1291.54966501  1291.54966501]
Logloss for the training set: 0.0531271680806
Logloss for the validation set: 0.0790080258239


## 5.3. Nested cross validation

Note that if you use the same data set to perform hyperparameter optimization and test set performance, your estimated test set performance may be over optimistic. To avoid this, you can perform a [nested CV](http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html).

This is similar to what I did in 5.1, but the difference is that `GridSearchCV` and `cross_val_score` gets a different CV iterator for nested CV. I think the main point is that the folds used to perform hyperparameter optimization and test set performance are different, which helps to avoid overfitting.

In [16]:
tuned_parameters = [{'C': [1, 10, 100, 1000, 10000, 100000]}]
model = linear_model.LogisticRegression(solver = 'sag', max_iter = 100, multi_class = 'multinomial')
cv_inner = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
cv_outer = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
clf = GridSearchCV(estimator = model, param_grid = tuned_parameters, cv=cv_inner, scoring='neg_log_loss')
clf.fit(df.iloc[:, 1:], df['y'])
print clf.best_params_
print clf.best_score_
scores = cross_val_score(clf, df.iloc[:, 1:], df['y'], scoring = 'neg_log_loss', cv = cv_outer, n_jobs = -1)
print scores
print scores.mean()

{'C': 100000}
-0.0766016063815
[-0.04647812 -0.03862962 -0.1681509  -0.06329839 -0.07018048]
-0.0773475013461


# 6. References

1. [\[Scikit-learn\] List of classes and functiones for scikit-learn](http://scikit-learn.org/stable/modules/classes.html)
2. [\[Scikit-learn\] 3. Model selection and evaluation](http://scikit-learn.org/stable/model_selection.html)
3. [\[Scikit-learn\] 3.1. Cross-validation: evaluating estimator performance](http://scikit-learn.org/stable/modules/cross_validation.html)
4. [\[Scikit-learn\] 3.2. Tuning the hyper-parameters of an estimator](http://scikit-learn.org/stable/modules/grid_search.html)
5. [\[Scikit-learn\] 3.3. Model evaluation: quantifying the quality of predictions](http://scikit-learn.org/stable/modules/model_evaluation.html)
6. [\[Scikit-learn\] Nested versus non-nested cross-validation](http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html)
7. [\[Towards Data Science\] Train/Test Split and Cross Validation in Python](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)